https://github.com/asahi417/tner

In [1]:
!pip install git+https://github.com/asahi417/tner

  Cloning https://github.com/asahi417/tner to /private/var/folders/z4/zq1sx9z918l822zf41yhflvr0000gn/T/pip-req-build-vjdju2_d
  Running command git clone -q https://github.com/asahi417/tner /private/var/folders/z4/zq1sx9z918l822zf41yhflvr0000gn/T/pip-req-build-vjdju2_d
     |████████████████████████████████| 13.9 MB 173 kB/s 
     |████████████████████████████████| 43 kB 5.2 MB/s 
     |████████████████████████████████| 48 kB 16.1 MB/s 
     |████████████████████████████████| 8.5 MB 43.4 MB/s 
     |████████████████████████████████| 64 kB 5.4 MB/s 
     |████████████████████████████████| 1.5 MB 34.0 MB/s 
     |████████████████████████████████| 102 kB 18.6 MB/s 
     |████████████████████████████████| 59 kB 4.1 MB/s 
     |████████████████████████████████| 2.4 MB 23.3 MB/s 
  Using cached Keras-2.4.3-py2.py3-none-any.whl (36 kB)
  Created wheel for tner: filename=tner-0.0.0-py3-none-any.whl size=25998 sha256=fd3bba8059fccf96cc5b3f584786433b0d803e763ad16e2a0a24fc9901856162
  Stored in d

# Use built-in NER dataset - BioNLP 2004 or BioCreative V CDR for medical dataset

In [2]:
import tner

In [3]:
tner.VALID_DATASET

['conll2003',
 'wnut2017',
 'ontonotes5',
 'mit_movie_trivia',
 'mit_restaurant',
 'fin',
 'bionlp2004',
 'wiki_ja',
 'wiki_news_ja',
 'bc5cdr',
 "panx_dataset/['ace', 'bg', 'da', 'fur', 'ilo', 'lij', 'mzn', 'qu', 'su', 'vi', 'af', 'bh', 'de', 'fy', 'io', 'lmo', 'nap', 'rm', 'sv', 'vls', 'als', 'bn', 'diq', 'ga', 'is', 'ln', 'nds', 'ro', 'sw', 'vo', 'am', 'bo', 'dv', 'gan', 'it', 'lt', 'ne', 'ru', 'szl', 'wa', 'an', 'br', 'el', 'gd', 'ja', 'lv', 'nl', 'rw', 'ta', 'war', 'ang', 'bs', 'eml', 'gl', 'jbo', 'map-bms', 'nn', 'sa', 'te', 'wuu', 'ar', 'ca', 'en', 'gn', 'jv', 'mg', 'no', 'sah', 'tg', 'xmf', 'arc', 'cbk-zam', 'eo', 'gu', 'ka', 'mhr', 'nov', 'scn', 'th', 'yi', 'arz', 'cdo', 'es', 'hak', 'kk', 'mi', 'oc', 'sco', 'tk', 'yo', 'as', 'ce', 'et', 'he', 'km', 'min', 'or', 'sd', 'tl', 'zea', 'ast', 'ceb', 'eu', 'hi', 'kn', 'mk', 'os', 'sh', 'tr', 'zh-classical', 'ay', 'ckb', 'ext', 'hr', 'ko', 'ml', 'pa', 'si', 'tt', 'zh-min-nan', 'az', 'co', 'fa', 'hsb', 'ksh', 'mn', 'pdc', 'simple', 'u

# Model fine tuning 
where transformers_model is a pre-trained model name from https://huggingface.co/models and dataset is a dataset alias (from built-in NER dataset) or path to custom dataset folder explained dataset section. Words tagged with O are outside of named entities and the I-XXX tag is used for words inside a named entity of type XXX. Whenever two entities of type XXX are immediately next to each other, the first word of the second entity will be tagged B-XXX in order to show that it starts another entity. The custom dataset should has train.txt and valid.txt file in a same folder.

cutom dataset example - 
train.txt - 

EU B-ORG

rejects O

German B-MISC

call O

to O

boycott O

British B-MISC

lamb O

. O


and test.txt and/or dev.txt

In [ ]:
from tner import TrainTransformersNER
trainer = TrainTransformersNER(
        dataset='bc5cdr',
        transformers_model='xlm-roberta-base',
        random_seed=1234,
        lr=1e-5,
        total_step=13000,
        warmup_step=700,
        weight_decay=1e-7,
        batch_size=16,
        batch_size_validation=2,
        max_seq_length=128,
        fp16=False,
        max_grad_norm=1.0
    )

In [ ]:
trainer.train() # gives u a model checkpoint

# Model testing

In [ ]:
from tner import TrainTransformersNER
# initialize model instance with model checkpoint
trainer = TrainTransformersNER(checkpoint='./ckpt/ontonote5_6bb4fdb286b5e32c068262c2a413639e', batch_size_validation=2)

In [ ]:
# Again, the test_dataset can be a path to custom dataset explained at
trainer.test(test_dataset='conll_2003') # test it on testing dataset
trainer.test(test_dataset='conll_2003', ignore_entity_type=True)

Entity span prediction: For better understanding of out-of-domain accuracy, we provide entity span prediction accuracy, which ignores the entity type and compute metrics only on the IOB entity position.

In [ ]:
trainer.test(test_dataset='conll2003', entity_span_prediction=True)

# Model Inference API

In [ ]:
import tner
classifier = tner.TransformersNER(checkpoint='path-to-checkpoint-dir')
test_sentences = [
    'I live in United States, but Microsoft asks me to move to Japan.',
    'I have an Apple computer.',
    'I like to eat an apple.'
]
classifier.predict(test_sentences)